In [1]:
import os
import cv2
import numpy as np
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib

In [2]:
data_loc = "compressor-data/"
files = os.listdir(data_loc)

files_len = len(files)
print(files_len)

MAX_FRAMES_PER_SECOND = 10

803


In [3]:
def format_timedelta(td):
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":","-")

In [4]:
def get_saving_frames_durations(cap, saving_fps):
    s = []
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)

    for i in np.arange(0, clip_duration, 1/saving_fps):
        s.append(i)
    
    return s

In [5]:
def video_analyze(file, override=False):
    video = cv2.VideoCapture(file)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total frames = " + str(total_frames))

    fps = video.get(cv2.CAP_PROP_FPS)
    print("FPS = " + str(fps))

In [6]:
file_name = data_loc + files[0]
video_analyze(file_name)

Total frames = 380
FPS = 10.0


In [7]:
def video_into_frames(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"

    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    saving_frames_per_second = min(fps, MAX_FRAMES_PER_SECOND)
    saving_frames_duration = get_saving_frames_durations(cap, saving_frames_per_second)

    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            break
            
        frame_duration = count/fps
        try:
            closest_duration = saving_frames_duration[0]
        except IndexError:
            break

        if frame_duration >= closest_duration:
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame)

            try:
                saving_frames_duration.pop(0)
            except IndexError:
                pass
        
        count += 1

In [8]:
video_into_frames(data_loc + "20210628-081000.03.mp4")

In [9]:
from IPython.display import Image

In [10]:
# contours, hier = cv2.findContours(im_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# result = cv2.imread("gray.png")

# bubbles = []
# for cnt in contours:
#     if 0<cv2.contourArea(cnt)<300:
#         cv2.drawContours(original_image, cnt, -1, (0,0,255), 2)
#         bubbles.append(cnt)


# cv2.imwrite("result.png", original_image)
# Image(filename="result.png")


In [11]:
def image_crop(image):
    black_mask = np.zeros_like(image)
    black_mask[:,:] = [255,255,255]

    # Cover top
    start_point = (0,0)
    end_point = (1280,90)
    color = (0,0,0)

    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    start_point = (0, 450)
    end_point = (1280, 720)
    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    return image

In [12]:
def grayscale_blur(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (7,7), 0)
    th, image = cv2.threshold(image, 127, 255, 0)

    return image

In [13]:
def blob_extraction(image, result):
    result = cv2.connectedComponentsWithStats(result, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = result
    
    for centroid in centroids:
        cv2.circle(image, (int(centroid[0]), int(centroid[1])), 7, (0, 255, 0), 1)
    
    return image

In [14]:
folder_path = "compressor-data/" + "20210628-081000.03-opencv/"
output_path = "compressor-data/" + "20210628-081000.03-opencv-output/"
image_list = sorted(os.listdir(folder_path))

for file in image_list:
    if str(file)[-3:] == "jpg":
        image = cv2.imread(folder_path + file)
        result = image_crop(image)
        result = grayscale_blur(result)
        image = blob_extraction(image, result)
        cv2.imwrite(os.path.join(output_path, file), image)

In [15]:
image_array = []
for file in sorted(os.listdir(output_path)):
    if str(file)[-3:] == "jpg":
        image = cv2.imread(output_path + file)
        height, width, layers = image.shape
        size = (width, height)
        image_array.append(image)

out = cv2.VideoWriter(output_path + 'result.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 10, size)

for i in range(len(image_array)):
    out.write(image_array[i])

out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
